# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [20]:
# Решено было взять mnist с keras, чтобы поинтереснее
from keras.datasets import mnist
def digits(num_training=49000, num_validation=1000, num_test=10000):
    (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
    X_train = X_train.reshape((*X_train.shape, 1))
    X_test = X_test.reshape((*X_test.shape, 1))
    X_train = np.asarray(X_train, dtype=np.float32)
    Y_train = np.asarray(Y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    Y_test = np.asarray(Y_test, dtype=np.int32).flatten()
    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    Y_val = Y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    Y_train = Y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    Y_test = Y_test[mask]
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel
    return X_train, Y_train, X_val, Y_val, X_test, Y_test
    

In [21]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = digits()

In [22]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, Y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, Y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, Y_test, batch_size=64)

In [23]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 28, 28, 1) (64,)
1 (64, 28, 28, 1) (64,)
2 (64, 28, 28, 1) (64,)
3 (64, 28, 28, 1) (64,)
4 (64, 28, 28, 1) (64,)
5 (64, 28, 28, 1) (64,)
6 (64, 28, 28, 1) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [6]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    #with tf.device(device):
    scores = model(x)
    print(scores.shape)
        
test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [7]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.conv1 = tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same")
        self.conv2 = tf.keras.layers.Conv2D(channel_2, 3, activation="relu", padding="same")
        self.fc1 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                  )
        self.flatten = tf.keras.layers.Flatten()

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        scores = self.fc1(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

In [8]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    #with tf.device(device):
    x = tf.zeros((64, 3, 32, 32))
    scores = model(x)
    print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [41]:
print_every=100

In [43]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    

    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    
    model = model_init_fn()
    optimizer = optimizer_init_fn()
    
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
    val_loss = tf.keras.metrics.Mean(name='val_loss')
    val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
    
    t = 0
    for epoch in range(num_epochs):
        
        # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
        train_loss.reset_states()
        train_accuracy.reset_states()
        
        for x_np, y_np in train_dset:
            with tf.GradientTape() as tape:
                
                # Use the model function to build the forward pass.
                scores = model(x_np, training=is_training)
                loss = loss_fn(y_np, scores)
    
                gradients = tape.gradient(loss, model.trainable_variables)
                optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                
                # Update the metrics
                train_loss.update_state(loss)
                train_accuracy.update_state(y_np, scores)
                
                if t % print_every == 0:
                    val_loss.reset_states()
                    val_accuracy.reset_states()
                    for test_x, test_y in val_dset:
                        # During validation at end of epoch, training set to False
                        prediction = model(test_x, training=False)
                        t_loss = loss_fn(test_y, prediction)

                        val_loss.update_state(t_loss)
                        val_accuracy.update_state(test_y, prediction)
                    
                    template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                    print (template.format(t, epoch+1,
                                         train_loss.result(),
                                         train_accuracy.result()*100,
                                         val_loss.result(),
                                         val_accuracy.result()*100))
                t += 1

In [10]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.947984218597412, Accuracy: 17.1875, Val Loss: 2.5949792861938477, Val Accuracy: 15.000000953674316
Iteration 100, Epoch 1, Loss: 0.660137414932251, Accuracy: 80.43006896972656, Val Loss: 0.5764584541320801, Val Accuracy: 79.9000015258789
Iteration 200, Epoch 1, Loss: 0.5209307074546814, Accuracy: 84.57711029052734, Val Loss: 0.4513089060783386, Val Accuracy: 86.19999694824219
Iteration 300, Epoch 1, Loss: 0.4622064530849457, Accuracy: 86.30606842041016, Val Loss: 0.42633676528930664, Val Accuracy: 87.19999694824219
Iteration 400, Epoch 1, Loss: 0.4166160523891449, Accuracy: 87.62858581542969, Val Loss: 0.37943366169929504, Val Accuracy: 89.60000610351562
Iteration 500, Epoch 1, Loss: 0.3917407989501953, Accuracy: 88.35765838623047, Val Loss: 0.3564143180847168, Val Accuracy: 88.80000305175781
Iteration 600, Epoch 1, Loss: 0.36796513199806213, Accuracy: 89.05730438232422, Val Loss: 0.3339751958847046, Val Accuracy: 90.20000457763672
Iteration 700, Epoch 1, 

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [24]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes) 

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.3678479194641113, Accuracy: 9.375, Val Loss: 2.326512098312378, Val Accuracy: 7.90000057220459
Iteration 100, Epoch 1, Loss: 0.7532694339752197, Accuracy: 79.06868743896484, Val Loss: 0.49550706148147583, Val Accuracy: 83.4000015258789
Iteration 200, Epoch 1, Loss: 0.5519068241119385, Accuracy: 84.42941284179688, Val Loss: 0.392255038022995, Val Accuracy: 87.69999694824219
Iteration 300, Epoch 1, Loss: 0.44783374667167664, Accuracy: 87.26121520996094, Val Loss: 0.2345581352710724, Val Accuracy: 93.0
Iteration 400, Epoch 1, Loss: 0.37554287910461426, Accuracy: 89.2183609008789, Val Loss: 0.21120353043079376, Val Accuracy: 93.0
Iteration 500, Epoch 1, Loss: 0.33091387152671814, Accuracy: 90.54078674316406, Val Loss: 0.17576709389686584, Val Accuracy: 94.4000015258789
Iteration 600, Epoch 1, Loss: 0.2955970764160156, Accuracy: 91.50114440917969, Val Loss: 0.19803622364997864, Val Accuracy: 93.80000305175781
Iteration 700, Epoch 1, Loss: 0.2689714729785919, Ac

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [25]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (28, 28, 1)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.1256823539733887, Accuracy: 14.0625, Val Loss: 2.5656204223632812, Val Accuracy: 17.100000381469727
Iteration 100, Epoch 1, Loss: 0.6335068941116333, Accuracy: 81.32734680175781, Val Loss: 0.5538548827171326, Val Accuracy: 81.80000305175781
Iteration 200, Epoch 1, Loss: 0.5027395486831665, Accuracy: 85.41667175292969, Val Loss: 0.4442679286003113, Val Accuracy: 86.9000015258789
Iteration 300, Epoch 1, Loss: 0.4461570978164673, Accuracy: 87.0431900024414, Val Loss: 0.41715800762176514, Val Accuracy: 88.20000457763672
Iteration 400, Epoch 1, Loss: 0.4032689034938812, Accuracy: 88.32605743408203, Val Loss: 0.3676564693450928, Val Accuracy: 89.20000457763672
Iteration 500, Epoch 1, Loss: 0.3803180754184723, Accuracy: 88.96269989013672, Val Loss: 0.3487617075443268, Val Accuracy: 90.10000610351562
Iteration 600, Epoch 1, Loss: 0.3574194014072418, Accuracy: 89.59286499023438, Val Loss: 0.33581745624542236, Val Accuracy: 90.80000305175781
Iteration 700, Epoch 1, 

Альтернативный менее гибкий способ обучения:

In [27]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, Y_train, batch_size=64, epochs=1, validation_data=(X_val, Y_val))
model.evaluate(X_test, Y_test)

313/313 [==============================] - 2s 6ms/step - loss: 0.2130 - sparse_categorical_accuracy: 0.9392


[0.21299292147159576, 0.9391999840736389]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [28]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    input_shape = (28,28,1)
    channel_1, channel_2, num_classes = 32, 16, 10
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same", input_shape=input_shape),
        tf.keras.layers.Conv2D(channel_2, 3, activation="relu", padding="same"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]
    model = tf.keras.Sequential(layers)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.323291301727295, Accuracy: 10.9375, Val Loss: 2.3317599296569824, Val Accuracy: 6.700000286102295
Iteration 100, Epoch 1, Loss: 1.8055198192596436, Accuracy: 52.10396194458008, Val Loss: 0.8948379755020142, Val Accuracy: 75.80000305175781
Iteration 200, Epoch 1, Loss: 1.171535849571228, Accuracy: 68.69558715820312, Val Loss: 0.5717899203300476, Val Accuracy: 83.0
Iteration 300, Epoch 1, Loss: 0.919281005859375, Accuracy: 75.09862518310547, Val Loss: 0.43958479166030884, Val Accuracy: 87.80000305175781
Iteration 400, Epoch 1, Loss: 0.7739182710647583, Accuracy: 78.85364532470703, Val Loss: 0.41879236698150635, Val Accuracy: 87.19999694824219
Iteration 500, Epoch 1, Loss: 0.6892250180244446, Accuracy: 81.0691146850586, Val Loss: 0.38131970167160034, Val Accuracy: 89.0999984741211
Iteration 600, Epoch 1, Loss: 0.6257354617118835, Accuracy: 82.7059097290039, Val Loss: 0.3902245759963989, Val Accuracy: 88.0
Iteration 700, Epoch 1, Loss: 0.5789628624916077, Accu

In [30]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, Y_train, batch_size=64, epochs=1, validation_data=(X_val, Y_val))
model.evaluate(X_test, Y_test)

313/313 [==============================] - 2s 7ms/step - loss: 0.1493 - sparse_categorical_accuracy: 0.9590


[0.14928683638572693, 0.9589999914169312]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [32]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    scores = model(x)
    print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


In [33]:
input_shape = (28, 28, 1)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.992885112762451, Accuracy: 10.9375, Val Loss: 2.7037649154663086, Val Accuracy: 17.200000762939453
Iteration 100, Epoch 1, Loss: 0.6523125767707825, Accuracy: 80.33724975585938, Val Loss: 0.5814943909645081, Val Accuracy: 81.5999984741211
Iteration 200, Epoch 1, Loss: 0.5184507369995117, Accuracy: 84.58488464355469, Val Loss: 0.4533456265926361, Val Accuracy: 86.9000015258789
Iteration 300, Epoch 1, Loss: 0.4599704146385193, Accuracy: 86.2853012084961, Val Loss: 0.41271519660949707, Val Accuracy: 87.0
Iteration 400, Epoch 1, Loss: 0.41557303071022034, Accuracy: 87.67534637451172, Val Loss: 0.3638101816177368, Val Accuracy: 90.20000457763672
Iteration 500, Epoch 1, Loss: 0.39124810695648193, Accuracy: 88.43251037597656, Val Loss: 0.35531482100486755, Val Accuracy: 90.5999984741211
Iteration 600, Epoch 1, Loss: 0.3677675426006317, Accuracy: 89.09109497070312, Val Loss: 0.3328782618045807, Val Accuracy: 91.0999984741211
Iteration 700, Epoch 1, Loss: 0.3499121

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

Опишите все эксперименты, результаты. Сделайте выводы.

In [47]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        self.conv1 = tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same")
        self.mp1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='valid')
        self.conv2 = tf.keras.layers.Conv2D(channel_2, 3, activation="relu", padding="same")
        self.mp2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='valid')
        self.fc1=tf.keras.layers.Dense(256, activation='relu')
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax')
        self.flatten = tf.keras.layers.Flatten()

        

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor)
        x = self.mp1(x)
        
        x = self.conv2(x)
        x = self.mp2(x)
        
        x=self.flatten(x)
        x=self.fc1(x)
        x=self.fc2(x)
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 7.5e-4
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.4646525382995605, Accuracy: 7.8125, Val Loss: 3.530057191848755, Val Accuracy: 22.5
Iteration 700, Epoch 1, Loss: 0.1361669898033142, Accuracy: 95.87419891357422, Val Loss: 0.09605340659618378, Val Accuracy: 97.5999984741211
Iteration 1400, Epoch 2, Loss: 0.045632120221853256, Accuracy: 98.57037353515625, Val Loss: 0.0678844228386879, Val Accuracy: 97.69999694824219
Iteration 2100, Epoch 3, Loss: 0.02690119855105877, Accuracy: 99.11576843261719, Val Loss: 0.07001487165689468, Val Accuracy: 97.89999389648438
Iteration 2800, Epoch 4, Loss: 0.0223290603607893, Accuracy: 99.2513656616211, Val Loss: 0.04552684724330902, Val Accuracy: 98.5
Iteration 3500, Epoch 5, Loss: 0.015188204124569893, Accuracy: 99.4708251953125, Val Loss: 0.06064394861459732, Val Accuracy: 98.5999984741211
Iteration 4200, Epoch 6, Loss: 0.01406104862689972, Accuracy: 99.51145935058594, Val Loss: 0.09125300496816635, Val Accuracy: 97.5999984741211
Iteration 4900, Epoch 7, Loss: 0.009468834

Хотелось бы отметить, что сети показывают хороший результат, идет уже борьба за лучшую точность. Как правило, 10 эпох никогда не требуется, и значения accuracy начинают колебаться просто то в одну, то в другую сторону, так что итоги будут смотреть по лучшим значениям, а не только в конце 10 эпохи

1) Для начала к трехслойной сети были добавлены слои maxpooling - ничего интересного

2) Добавим два блока Conv2d, затем maxpool, затем повторим - обучаться стало куда тяжелее, результат особо лучше не стал

3) Поставим увеличение карт - 16, 32, 64 - Результат примерно такой же, но насколько же дольше мы стали обучаться

4) Сразу несколько изменений. Все ядра сверток фиксированный размер 3 на 3, maxpooling  с valid, еще один полносвязный слой на 256 нейронов, уменьшили темп обучения, оставили только по одной свертке на уровень - на пятой эпохе 98.6 и это лучший результат, что удалось получить(он в ячейке)

5) Попутно введем дропаут и батчнорм(просто ради интереса, ничем не обоснованно) - стало хуже, но занимательно, что при дропауте после последней свертки значения accuracy на val, было куда выше, чем на train.

6) Добавляем еще два полносвязных слоя - и все стало хуже, точность около 93, явное переобучение.


Сеть в ячейке лучшая с достигнутой accuracy на val(5 эпоха), и лучшая с точки зрения вычисления/результаты. 

Для остановки на лучшей эпохе, можно использовать keras.callback.earlystopping, но в текущей реализации его не очень удобно прикрутить, так что отслеживание на глазок

Еще раз хотелось бы подчеркнуть, что абсолютно все модели выдавали 97% на val(на лучших эпохах)(кроме 5) и соревнование шло за лучшую точность

Судя по kaggle,  рецепт лучшей accuracy - агументация данных, по две свертки одинаково размера, после которых следует maxpool,  и батчнорм, число фильтров 64, 128б 256,  полносвязный слой на 512 нейронов, 50 эпох - дают 99+ на val

